In [1]:
###getting the reports local url's:###
##extracting folders id from shared memory

from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd

# Authenticate
auth.authenticate_user()
drive_service = build('drive', 'v3')

folder_id = "1e0eA-AIsz_BSwVHOppJMXECX42hBfG4J"

def list_all_files_in_folder_recursive(parent_id, parent_path=""):
    """Recursively list all files in a folder and subfolders"""
    all_files = []

    query = f"'{parent_id}' in parents and trashed=false"
    page_token = None

    while True:
        response = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token
        ).execute()

        for item in response.get('files', []):
            item_path = f"{parent_path}/{item['name']}" if parent_path else item['name']

            if item['mimeType'] == 'application/vnd.google-apps.folder':
                # Recurse into subfolder
                all_files.extend(list_all_files_in_folder_recursive(item['id'], item_path))
            else:
                all_files.append({
                    "file_name": item['name'],
                    "file_path": item_path,
                    "file_id": item['id'],
                    "file_url": f"https://drive.google.com/file/d/{item['id']}/view?usp=sharing"
                })

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return all_files

# Run the recursive function
files_list = list_all_files_in_folder_recursive(folder_id)

# Convert to DataFrame
df_files = pd.DataFrame(files_list)
##df_files.head()


##getting local id of the file
!pip install python-docx
import os
from googleapiclient.http import MediaIoBaseDownload
import io

def get_chapter_file(chapter_num: int, year: str, download_dir="/content") -> str:
    """
    Download a chapter file for a given year from Google Drive and return its local path.

    Args:
        chapter_num (int): The chapter number (e.g., 1, 2, ..., 14).
        year (str): The year folder name.
        download_dir (str): Local folder to save the file. Defaults to /content in Colab.

    Returns:
        str: Local path to the downloaded file, or None if not found.
    """
    # Normalize chapter filename (01, 02, ...)
    chapter_str = f"{chapter_num:02}"

    # Find matching file in df_files
    match = df_files[
        (df_files["file_name"].str.contains(chapter_str)) &
        (df_files["file_path"].str.contains(f"{year}/"))
    ]

    if match.empty:
        print(f"❌ No file found for chapter {chapter_str} in year {year}")
        return None

    file_id = match.iloc[0]["file_id"]
    file_name = match.iloc[0]["file_name"]
    local_path = os.path.join(download_dir, year, file_name)

    # Ensure year directory exists locally
    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    # Download the file from Google Drive
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(local_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        if status:
            print(f"⬇️ Download {int(status.progress() * 100)}%.")

    print(f"✅ Saved to {local_path}")
    return local_path

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.6 MB/s eta 0:00:00


In [86]:
#a code that downloads the files to a directory called content/reports
import os
import io
from googleapiclient.http import MediaIoBaseDownload

def download_all_chapters(download_dir="/content/reports", years=range(2023, 2024), chapters=range(1, 16)):
    """
    Downloads all chapters for all years to local environment.

    Returns:
        dict: {year: {chapter_number: local_path}}
    """
    all_paths = {}

    for year in years:
        year_str = str(year)
        all_paths[year_str] = {}

        for chapter in chapters:
            chapter_str = f"{chapter:02}"

            # Find matching file in df_files
            match = df_files[
                (df_files["file_name"].str.contains(chapter_str)) &
                (df_files["file_path"].str.contains(f"{year_str}/"))
            ]

            if match.empty:
                print(f"⚠️ Chapter {chapter_str} not found for year {year_str}")
                continue

            file_id = match.iloc[0]["file_id"]
            file_name = match.iloc[0]["file_name"]
            local_path = os.path.join(download_dir, year_str, f"{chapter_str}_{file_name}")

            # Ensure folder exists
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            # Download file
            request = drive_service.files().get_media(fileId=file_id)
            fh = io.FileIO(local_path, "wb")
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()
                if status:
                    print(f"⬇️ Download {int(status.progress() * 100)}% for {file_name}")

            print(f"✅ Saved {file_name} to {local_path}")
            all_paths[year_str][chapter] = local_path

    return all_paths
download_all_chapters()

⬇️ Download 100% for 01.docx
✅ Saved 01.docx to /content/reports/2023/01_01.docx
⬇️ Download 100% for 02.docx
✅ Saved 02.docx to /content/reports/2023/02_02.docx
⬇️ Download 100% for 03.docx
✅ Saved 03.docx to /content/reports/2023/03_03.docx
⬇️ Download 100% for 04.docx
✅ Saved 04.docx to /content/reports/2023/04_04.docx
⬇️ Download 100% for 05.docx
✅ Saved 05.docx to /content/reports/2023/05_05.docx
⬇️ Download 100% for 06.docx
✅ Saved 06.docx to /content/reports/2023/06_06.docx
⬇️ Download 100% for 07.docx
✅ Saved 07.docx to /content/reports/2023/07_07.docx
⬇️ Download 100% for 08.docx
✅ Saved 08.docx to /content/reports/2023/08_08.docx
⬇️ Download 100% for 09.docx
✅ Saved 09.docx to /content/reports/2023/09_09.docx
⬇️ Download 100% for 10.docx
✅ Saved 10.docx to /content/reports/2023/10_10.docx
⬇️ Download 100% for 11.docx
✅ Saved 11.docx to /content/reports/2023/11_11.docx
⬇️ Download 100% for 12.docx
✅ Saved 12.docx to /content/reports/2023/12_12.docx
⬇️ Download 100% for 13.docx

{'2023': {1: '/content/reports/2023/01_01.docx',
  2: '/content/reports/2023/02_02.docx',
  3: '/content/reports/2023/03_03.docx',
  4: '/content/reports/2023/04_04.docx',
  5: '/content/reports/2023/05_05.docx',
  6: '/content/reports/2023/06_06.docx',
  7: '/content/reports/2023/07_07.docx',
  8: '/content/reports/2023/08_08.docx',
  9: '/content/reports/2023/09_09.docx',
  10: '/content/reports/2023/10_10.docx',
  11: '/content/reports/2023/11_11.docx',
  12: '/content/reports/2023/12_12.docx',
  13: '/content/reports/2023/13_13.docx',
  14: '/content/reports/2023/14_14.docx'}}

In [ ]:
import os
import json
import pandas as pd
from docx import Document
!apt-get install -y libreoffice
import subprocess

def convert_doc_to_docx(base_dir="/content/reports"):
    for root, _, files in os.walk(base_dir):
        for fname in files:
            if fname.endswith(".doc") and not fname.endswith(".docx"):
                fpath = os.path.join(root, fname)
                print(fpath)
                subprocess.run([
                    "libreoffice", "--headless", "--convert-to", "docx", fpath, "--outdir", root
                ])

# convert all docs
convert_doc_to_docx("/content/reports")

In [ ]:
# very initial code - does not working. does extracting the json correctly

#import os
# import subprocess
# from docx import Document

# def convert_doc_to_docx(doc_path):
#     """
#     Convert a .doc file to .docx using LibreOffice (headless mode)
#     Returns the path to the converted .docx file
#     """
#     if not doc_path.endswith(".doc"):
#         return doc_path  # already .docx

#     converted_path = doc_path + "x"  # simple .docx path
#     os.makedirs(os.path.dirname(converted_path), exist_ok=True)

#     # LibreOffice command-line conversion
#     subprocess.run([
#         "libreoffice",
#         "--headless",
#         "--convert-to",
#         "docx",
#         "--outdir",
#         os.path.dirname(doc_path),
#         doc_path
#     ], check=True)

#     return converted_path

# def convert_wmf_to_png(wmf_path, png_path):
#     """Convert .wmf or .x-wmf to .png using ImageMagick"""
#     subprocess.run([
#         "convert", wmf_path, png_path
#     ], check=True)

# def extract_images_from_docx(docx_path, output_dir):
#     """
#     Extracts all images from a .docx file and saves them in output_dir.
#     WMF images are automatically converted to PNG.
#     """
#     doc = Document(docx_path)
#     image_count = 0
#     for i, rel in enumerate(doc.part._rels):
#         rel = doc.part._rels[rel]
#         if "image" in rel.target_ref:
#             image_count += 1
#             img_data = rel.target_part.blob
#             ext = rel.target_part.content_type.split('/')[-1]
#             if ext == 'jpeg':
#                 ext = 'jpg'

#             img_name = f"image_{i+1}.{ext}"
#             img_path = os.path.join(output_dir, img_name)
#             os.makedirs(output_dir, exist_ok=True)
#             with open(img_path, "wb") as f:
#                 f.write(img_data)

#             # Convert WMF to PNG
#             if ext in ["x-wmf", "wmf"]:
#                 png_path = os.path.splitext(img_path)[0] + ".png"
#                 try:
#                     convert_wmf_to_png(img_path, png_path)
#                     os.remove(img_path)  # remove original WMF
#                     print(f"🔄 Converted {img_path} → {png_path}")
#                 except Exception as e:
#                     print(f"❌ Failed to convert {img_path}: {e}")

#     return image_count

# # Make sure LibreOffice & ImageMagick are installed
# !apt-get update
# !apt-get install -y libreoffice imagemagick

# # Paths
# reports_dir = "/content/reports"          # local folder with reports
# images_output_dir = "/content/reports_images"

# years = [str(y) for y in range(2001, 2025)]
# chapters = list(range(1, 16))  # chapters 1-15

# for year in years:
#     for chapter in chapters:
#         chapter_str = f"{chapter:02}"
#         year_folder = os.path.join(reports_dir, year)
#         if not os.path.exists(year_folder):
#             continue

#         # Find all .doc and .docx files for this chapter
#         chapter_files = [f for f in os.listdir(year_folder)
#                          if f.startswith(chapter_str) and (f.endswith(".docx") or f.endswith(".doc"))]

#         if not chapter_files:
#             print(f"⚠️ No Word files found for year {year}, chapter {chapter_str}")
#             continue

#         for file_name in chapter_files:
#             file_path = os.path.join(year_folder, file_name)
#             # Convert .doc to .docx if needed
#             if file_path.endswith(".doc"):
#                 print(f"🔄 Converting {file_path} to .docx")
#                 try:
#                     file_path = convert_doc_to_docx(file_path)
#                 except Exception as e:
#                     print(f"❌ Conversion failed for {file_path}: {e}")
#                     continue

#             out_dir = os.path.join(images_output_dir, year, chapter_str)
#             count = extract_images_from_docx(file_path, out_dir)
#             print(f"🖼 Extracted {count} images from {file_path} to {out_dir}")


In [ ]:
from docx import Document
import os

docx_path = "/content/reports/2001/02_02.docx"
output_dir = "images_from_docx"
os.makedirs(output_dir, exist_ok=True)

doc = Document(docx_path)

count = 0
for rel in doc.part._rels.values():
    if "image" in rel.target_ref:
        img_data = rel.target_part.blob
        img_ext = os.path.splitext(rel.target_ref)[1]
        count += 1
        with open(os.path.join(output_dir, f"image_{count}{img_ext}"), "wb") as f:
            f.write(img_data)

print(f"Extracted {count} images to '{output_dir}'")

!apt-get install -y imagemagick
!pip install Wand

from wand.image import Image
import os

wmf_path = "images_from_docx/image_2.wmf"
png_path = "images_from_docx/image_2.png"

with Image(filename=wmf_path) as img:
    img.format = 'png'
    img.save(filename=png_path)

print("Converted WMF to PNG!")



In [ ]:
##extracting all the photos from a docx file into a given directory

import os
from docx import Document
from wand.image import Image

##extracting images by file name format is ok with 2001-2016
def extract_and_convert_images(docx_path, output_dir, base_name="image"):
    """
    Extracts all images from a DOCX file and converts WMF to PNG.

    Parameters:
        docx_path (str): Path to the DOCX file.
        output_dir (str): Directory to save extracted images.
        base_name (str): Base name for saved images.
    """
    os.makedirs(output_dir, exist_ok=True)
    doc = Document(docx_path)
    count = 0

    for rel in doc.part._rels.values():
        if "image" in rel.target_ref:
            img_data = rel.target_part.blob
            img_ext = os.path.splitext(rel.target_ref)[1].lower()
            count += 1
            img_filename = os.path.join(output_dir, f"{base_name}_{count}{img_ext}")

            # Save the original image first
            with open(img_filename, "wb") as f:
                f.write(img_data)

            # Convert WMF to PNG
            if img_ext == ".wmf":
                png_filename = os.path.join(output_dir, f"{base_name}_{count}.png")
                with Image(filename=img_filename) as img:
                    img.format = 'png'
                    img.save(filename=png_filename)
                os.remove(img_filename)  # optional: remove original WMF
                print(f"Converted WMF to PNG: {png_filename}")
            else:
                print(f"Saved image: {img_filename}")

    print(f"Done! Extracted {count} images to '{output_dir}'.")

# Example usage
!apt-get install -y imagemagick
!pip install Wand

docx_file = "/content/reports/2016/02_02.docx"
save_dir = "/content/images_from_docx2"
extract_and_convert_images(docx_file, save_dir, base_name="2001_02_")


In [ ]:
###code that extract the images if they hare in a table which contains the word "תרשים"

from docx import Document
import os
from wand.image import Image
from docx import Document
import os
from wand.image import Image

graph_summary = {}
def extract_images_from_docx_tables(docx_path, output_dir, chapter, year, base_name="image"):
    os.makedirs(output_dir, exist_ok=True)
    doc = Document(docx_path)
    count = 0

    def save_image(img_data, img_ext, idx):
        img_filename = os.path.join(output_dir, f"{base_name}_{idx}{img_ext}")
        with open(img_filename, "wb") as f:
            f.write(img_data)

        # Convert WMF to PNG
        if img_ext == ".wmf":
            png_filename = os.path.join(output_dir, f"{base_name}_{idx}.png")
            with Image(filename=img_filename) as img:
                img.format = 'png'
                img.save(filename=png_filename)
            os.remove(img_filename)
            return png_filename
        return img_filename

    # Iterate over tables
    for table in doc.tables:
        # Get full text of the first row (all cells combined)
        first_row_text = " | ".join(cell.text.strip() for cell in table.rows[0].cells)
        if not "תרשים" in first_row_text:
           continue
        found_image = False
        for row in table.rows:
            for cell in row.cells:
                for para in cell.paragraphs:
                    text_before = para.text.strip()
                    for run in para.runs:
                        # Look for inline images
                        for blip in run._element.xpath(".//a:blip"):
                            rId = blip.get("{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed")
                            img_part = doc.part.related_parts[rId]
                            img_data = img_part.blob
                            img_ext = os.path.splitext(img_part.partname)[1].lower()
                            count += 1
                            img_filename = save_image(img_data, img_ext, count)
                            ##print(f"Saved image: {img_filename}, text before: '{text_before}'")
                            found_image = True
                            graph_summary[f'{count}_{chapter}_{year}']= first_row_text
                            break
                        if found_image:
                            break
                    if found_image:
                        break
                if found_image:
                    break
            if found_image:
                break


base_dir = "/content/reports"
save_dir = "/content/graphs_2001-2016"

for year in range(2001, 2017):  # 2001–2016 inclusive
    year_folder = os.path.join(base_dir, str(year))

    for chapter in range(1, 17):  # 01–16
        chapter_str = f"{chapter:02}"

        # Construct filename like "02_02.docx"
        docx_file = os.path.join(year_folder, f"{chapter_str}_{chapter_str}.docx")
        if not os.path.exists(docx_file):
            continue  # skip missing files

        # Output directory for this chapter
        out_path = os.path.join(save_dir, str(year), chapter_str)
        os.makedirs(out_path, exist_ok=True)

        # Base name for images, e.g. "2001_02_"
        base_name = f"{year}_{chapter_str}_"

        extract_images_from_docx_tables(docx_file, out_path,chapter,year, base_name=base_name)
        print("finish extracting for year: ", year, "chapter: ", chapter)

# Save JSON mapping
with open(os.path.join("/content/graphs_2001-2016", "graphs_summary.json"), "w", encoding="utf-8") as f:
    json.dump(graph_summary, f, ensure_ascii=False, indent=2)
print(f"Done! Extracted {count} images to '{output_dir}'.")


In [ ]:
##code that extracting files to drive

from google.colab import drive
import shutil

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Create a zip of your folder
shutil.make_archive("graphs_2001-2016", 'zip', "graphs_2001-2016")

# 3️⃣ Copy the zip into Google Drive (choose your path inside "My Drive")
shutil.move("graphs_2001-2016.zip", "/content/drive/My Drive/graphs_2001-2016.zip")

print("✅ File saved to Google Drive: /content/drive/My Drive/graphs_2001-2016.zip")


Mounted at /content/drive
✅ File saved to Google Drive: /content/drive/My Drive/graphs_2001-2016.zip


In [6]:
## extracting xlsx files correctly
import os
import zipfile

def extract_embedded_excels(docx_path, output_dir="/content/extracted_files"):
    """
    Extract embedded Excel files from a .docx into a local directory.

    Args:
        docx_path (str): Path to the .docx file
        output_dir (str): Directory where extracted .xlsx files will be saved
    """
    os.makedirs(output_dir, exist_ok=True)

    with zipfile.ZipFile(docx_path, "r") as docx:
        # All embedded objects are inside word/embeddings/
        for file_name in docx.namelist():
            if file_name.startswith("word/embeddings/") and file_name.endswith(".xlsx"):
                # Create a unique name using the docx file's base name
                base_name = os.path.splitext(os.path.basename(docx_path))[0]
                out_file = os.path.join(output_dir, f"{base_name}_{os.path.basename(file_name)}")

                with docx.open(file_name) as source, open(out_file, "wb") as target:
                    target.write(source.read())
                print(f"Extracted: {out_file}")


extract_embedded_excels("/content/reports/2023/01_01.docx")

Extracted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet.xlsx
Extracted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet1.xlsx
Extracted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet2.xlsx
Extracted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet3.xlsx
Extracted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet4.xlsx


In [9]:
import os
import zipfile
import shutil
import pandas as pd

def extract_excels_to_csv(docx_path, output_dir="/content/extracted_files"):
    """
    Extract embedded Excel files from a .docx into a clean directory and convert them to CSV.

    Args:
        docx_path (str): Path to the .docx file
        output_dir (str): Directory where extracted .csv files will be saved
    """
    # Reset the output directory (erase if exists)
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    with zipfile.ZipFile(docx_path, "r") as docx:
        for file_name in docx.namelist():
            if file_name.startswith("word/embeddings/") and file_name.endswith(".xlsx"):
                base_name = os.path.splitext(os.path.basename(docx_path))[0]
                excel_temp = os.path.join(output_dir, f"{base_name}_{os.path.basename(file_name)}")

                # Save the Excel file temporarily
                with docx.open(file_name) as source, open(excel_temp, "wb") as target:
                    target.write(source.read())

                # Convert Excel → CSV (one CSV per sheet if multiple sheets exist)
                try:
                    xls = pd.ExcelFile(excel_temp)
                    for sheet in xls.sheet_names:
                        df = pd.read_excel(excel_temp, sheet_name=sheet)
                        csv_name = f"{os.path.splitext(excel_temp)[0]}_{sheet}.csv"
                        df.to_csv(csv_name, index=False)
                        print(f"Converted: {csv_name}")
                except Exception as e:
                    print(f"⚠️ Could not convert {excel_temp} to CSV: {e}")

                # Remove the temporary Excel file
                os.remove(excel_temp)

extract_excels_to_csv("/content/reports/2023/01_01.docx")

Converted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet_גיליון1.csv
Converted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet1_גיליון1.csv
Converted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet2_גיליון1.csv
Converted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet3_גיליון1.csv
Converted: /content/extracted_files/01_01_Microsoft_Excel_Worksheet4_גיליון1.csv


In [102]:
## extracting the names on the tables that it is in and naming the files correctly
import os
import shutil
import zipfile
import pandas as pd
import json
from docx import Document

def extract_docx_tables_and_excels(docx_path, year, chapter, output_base="/content/extracted_files"):
    """
    Extract embedded Excel files from a DOCX, convert to CSV, and map table headers to them.

    Args:
        docx_path (str): Path to the .docx file
        year (str|int): Year to tag output files
        chapter (str|int): Chapter number to tag output files
        output_base (str): Base output directory
    """
    year, chapter = str(year), str(chapter)
    output_dir = os.path.join(output_base, year, chapter)

    # Reset output dir
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # ---------- Pass 1: Collect names from tables ----------
    # doc = Document(docx_path)
    # table_headers = []

    # for table in doc.tables:
    #     # Skip tables with no rows or no first cell text
    #     if len(table.rows) < 2:
    #         continue

    #     # Get the header text from the first row (row 0) if needed
    #     header_text = table.rows[0].cells[0].text.strip()
    #     if "תרשים" not in header_text:
    #         continue

    #     # Check only the second row (index 1)
    #     second_row = table.rows[1]
    #     has_content = False
    #     for cell in second_row.cells:
    #         # Check if the cell has any content (text, image, object)
    #         if len(cell._element) > 0 or cell.text.strip():
    #             print(len(cell._element))
    #             has_content = True
    #             print("has content: ",header_text)
    #             break

    # if not has_content:
    #     continue  # skip tables with an empty second row
    #     print("does not has content but do has 2+ cols: ", header_text)

    # # If second row has content, add the header
    # table_headers.append(header_text)
    doc = Document(docx_path)
    table_headers = []
    for table in doc.tables:
        if table.rows and table.cell(0,0).text:  # header exists
            if len(table.rows)==1:
              continue
            header_text = table.rows[0].cells[0].text.strip()
            if "תרשים" in header_text:
                # Repeat header as many times as there are columns
                #for _ in range(len(table.columns)):
                table_headers.append(header_text)

    print("Relevant table headers:", table_headers)

    # ---------- Pass 2: Extract embedded Excels and save as CSV ----------
    serial = 0
    csv_files = []
    with zipfile.ZipFile(docx_path, "r") as docx:
        for file_name in docx.namelist():
            if file_name.startswith("word/embeddings/") and file_name.endswith(".xlsx"):
                serial += 1
                out_csv = os.path.join(output_dir, f"{serial}_{chapter}_{year}.csv")

                # Save Excel temp
                excel_temp = os.path.join(output_dir, f"temp_{serial}.xlsx")
                with docx.open(file_name) as source, open(excel_temp, "wb") as target:
                    target.write(source.read())

                # Convert Excel -> CSV
                try:
                    xls = pd.ExcelFile(excel_temp)
                    # if multiple sheets -> concatenate into one CSV
                    df_all = []
                    for sheet in xls.sheet_names:
                        df = pd.read_excel(excel_temp, sheet_name=sheet)
                        df_all.append(df)
                    df_full = pd.concat(df_all, ignore_index=True)
                    df_full.to_csv(out_csv, index=False)
                    csv_files.append(out_csv)
                    print(f"Extracted: {out_csv}")
                except Exception as e:
                    print(f"⚠️ Could not convert {excel_temp}: {e}")
                finally:
                    os.remove(excel_temp)

    # ---------- Pass 3: Build JSON mapping ----------
    mapping = {}
    for idx, (csv_path, header) in enumerate(zip(csv_files, table_headers), start=1):
        key = f"{idx}_{chapter}_{year}"
        mapping[key] = header

    json_path = os.path.join(output_dir, f"{chapter}_{year}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(mapping, f, ensure_ascii=False, indent=2)

    # ---------- Sanity check ----------
    print("\n--- Sanity Check ---")
    print(f"Headers found: {len(table_headers)}")
    print(f"CSVs extracted: {len(csv_files)}")
    if len(table_headers) == len(csv_files):
        print("✅ Counts match.")
    else:
        print("⚠️ Mismatch between headers and CSVs!")

    print(f"\nJSON mapping saved at: {json_path}")
    return mapping



# years= [2019,2021,2022,2023]
# for year in years:
#   for ch in range(1,15):
#     extract_docx_tables_and_excels(f"/content/reports/{year}/{ch:02}_{ch:02}.docx",year,ch)

extract_docx_tables_and_excels("/שנתון מלא סופי.docx" ,2024,1)

Relevant table headers: ["תרשים 1א': התפלגות הילדים בישראל, לפי דת (אחוזים) / 2023", "תרשים 1ב': יישובים* בהם ילדים מנו כ-25% ופחות או כ-50% ויותר מאוכלוסיית היישוב* / 2023", "תרשים 1ג': מספר הילדים (17-0) שקיבלו במהלך השנה מעמד של תושב ארעי או תושב קבע / 2020-2018", "תרשים 1ד': שיעור הילדים (17-0) הזרים שקיבלו במהלך השנה מעמד של תושב ארעי, לפי ארץ מוצא / 2020-2018", "תרשים 2א': שיעור הילדים העולים* מכלל אוכלוסיית העולים**, לפי תקופת עלייה / 2023-1948", 'תרשים 2ב\': מספר הילדים העולים (0-17) ילידי חו"ל לפי שנים לפי שנת עלייה / 2010 -\n2023', "תרשים 2ג': גיל הילדים (0-17) שעלו במהלך שנת 2023 (אחוזים)", 'תרשים 2ד\': שיעור הילדים העולים, ילידי חו"ל, לפי ארץ מוצא / 2023', "תרשים 2ה': מספר הילדים העולים* וילדי עולים** בגילאי 17-0 / 2008-2023***", "תרשים 2ו': התפלגות ילדי העולים* (17-0) וחלקם מכלל הילדים בישראל /\n2023-2010**", "תרשים 2ז': שיעור ילדי עולים*, לפי מספר הילדים במשפחה / 2023-2017", "תרשים 2ח': מספר ילדים* ממוצע במשפחות עולים עם ילדים בהשוואה לכלל המשפחות בישראל / 2023-2010**", "

{'1_1_2024': "תרשים 1א': התפלגות הילדים בישראל, לפי דת (אחוזים) / 2023",
 '2_1_2024': "תרשים 1ב': יישובים* בהם ילדים מנו כ-25% ופחות או כ-50% ויותר מאוכלוסיית היישוב* / 2023",
 '3_1_2024': "תרשים 1ג': מספר הילדים (17-0) שקיבלו במהלך השנה מעמד של תושב ארעי או תושב קבע / 2020-2018",
 '4_1_2024': "תרשים 1ד': שיעור הילדים (17-0) הזרים שקיבלו במהלך השנה מעמד של תושב ארעי, לפי ארץ מוצא / 2020-2018",
 '5_1_2024': "תרשים 2א': שיעור הילדים העולים* מכלל אוכלוסיית העולים**, לפי תקופת עלייה / 2023-1948",
 '6_1_2024': 'תרשים 2ב\': מספר הילדים העולים (0-17) ילידי חו"ל לפי שנים לפי שנת עלייה / 2010 -\n2023',
 '7_1_2024': "תרשים 2ג': גיל הילדים (0-17) שעלו במהלך שנת 2023 (אחוזים)",
 '8_1_2024': 'תרשים 2ד\': שיעור הילדים העולים, ילידי חו"ל, לפי ארץ מוצא / 2023',
 '9_1_2024': "תרשים 2ה': מספר הילדים העולים* וילדי עולים** בגילאי 17-0 / 2008-2023***",
 '10_1_2024': "תרשים 2ו': התפלגות ילדי העולים* (17-0) וחלקם מכלל הילדים בישראל /\n2023-2010**",
 '11_1_2024': "תרשים 2ז': שיעור ילדי עולים*, לפי מספר הילדים

In [41]:
# import zipfile
# from docx import Document
# import xml.etree.ElementTree as ET

# def count_embedded_excels(docx_path):
#     """
#     Count embedded Excel files (.xlsx or .bin) in tables whose header contains 'תרשים'.
#     Returns a mapping of table serial number -> number of embedded Excel objects.
#     """
#     doc = Document(docx_path)

#     # Read DOCX relationships to map rId -> target file
#     with zipfile.ZipFile(docx_path, "r") as z:
#         rels_xml = z.read("word/_rels/document.xml.rels")
#         embedded_files = [f for f in z.namelist() if f.startswith("word/embeddings/")]

#     rels_tree = ET.fromstring(rels_xml)
#     rid_to_target = {rel.attrib['Id']: rel.attrib['Target'] for rel in rels_tree}

#     table_counts = {}
#     table_serial = 0

#     for table in doc.tables:
#         if table.rows and table.cell(0,0).text:
#             header_text = table.rows[0].cells[0].text.strip()
#             if "תרשים" in header_text:
#                 table_serial += 1
#                 count = 0
#                 for elem in table._element.iter():
#                     tag_name = elem.tag.split('}')[-1].lower()  # strip namespace
#                     if tag_name in ['object', 'oleobject']:
#                         print("hi")
#                         rid = elem.attrib.get("{http://schemas.openxmlformats.org/officeDocument/2006/relationships}id")
#                         print(rid)
#                         if rid and rid in rid_to_target:
#                             target = rid_to_target[rid]
#                             if target.lower().endswith(".xlsx") or target.lower().endswith(".bin"):
#                                 count += 1
#                 table_counts[table_serial] = count

#     print("\nTable serial number -> embedded Excel (.xlsx or .bin) count:")
#     for serial, count in table_counts.items():
#         print(f"Table {serial}: {count}")

#     print(f"\nTotal embedded files in document (from ZIP): {len(embedded_files)}")
#     return table_counts

# # Example usage
# table_counts = count_embedded_excels("/content/reports/2023/01_01.docx")



Table serial number -> embedded Excel (.xlsx or .bin) count:
Table 1: 0
Table 2: 0
Table 3: 0
Table 4: 0
Table 5: 0

Total embedded files in document (from ZIP): 5


In [57]:
# import zipfile
# from docx import Document

# def count_images_in_tables(docx_path):
#     """
#     Count all images (pictures) in tables whose header contains 'תרשים'.
#     Returns a mapping: table_serial -> number of images.
#     """
#     doc = Document(docx_path)
#     table_serial = 0
#     table_image_counts = {}

#     def scan_for_images(element):
#         count = 0
#         for elem in element.iter():
#             tag_name = elem.tag.split('}')[-1].lower()
#             if tag_name == "blip":  # <a:blip> is the image reference
#                 print("hi")
#                 rid = elem.attrib.get("{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed")
#                 if rid:
#                     count += 1
#         return count

#     for table in doc.tables:
#         if table.rows and table.cell(0,0).text:
#             header_text = table.rows[0].cells[0].text.strip()
#             if "תרשים" in header_text:
#                 table_serial += 1
#                 count = scan_for_images(table._element)
#                 table_image_counts[table_serial] = count

#     # Print results
#     for serial, count in table_image_counts.items():
#         print(f"Table {serial}: {count} images")

#     total_images = sum(table_image_counts.values())
#     print(f"\nTotal images in document tables with 'תרשים': {total_images}")

#     return table_image_counts

# # Example usage
# table_image_counts = count_images_in_tables("/content/reports/2023/01_01.docx")


Table 1: 0 images
Table 2: 0 images
Table 3: 0 images
Table 4: 0 images
Table 5: 0 images

Total images in document tables with 'תרשים': 0


In [107]:
from google.colab import drive
import shutil
import os

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Define folder to zip and output zip path
folder_to_zip = "/content/extracted_files"  # make sure this exists
zip_name = "/content/extracted_files"  # zip will become extracted_files.zip

# 3️⃣ Create the zip
shutil.make_archive(zip_name, 'zip', folder_to_zip)

# 4️⃣ Move zip into Google Drive
drive_path = "/content/drive/My Drive/excel_and_json_2019_2021_2024.zip"
shutil.move(zip_name + ".zip", drive_path)

print(f"✅ File saved to Google Drive: {drive_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ File saved to Google Drive: /content/drive/My Drive/excel_and_json_2019_2021_2024.zip
